In [57]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
import pickle
#import seaborn as sns
#import matplotlib.pyplot as plt
from collections import Counter
#scikit learn  library
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
#Use LogisticRegression for classifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
#oversampling

import warnings
warnings.filterwarnings('ignore')


In [0]:
data=pd.read_csv('/content/drive/My Drive/Colab Notebooks/extra/train.csv')
test=pd.read_csv('/content/drive/My Drive/Colab Notebooks/extra/test.csv')


In [0]:
Y=pd.DataFrame(data['gender'])  # seperate dataframe for dpenedent variable
data=data.drop(['gender'],axis=1) #data frame of independent feature

In [61]:
test.head()

,session_id,startTime,endTime,ProductList
0,u12112,08/12/14 13:36,08/12/14 13:36,A00002/B00003/C00006/D19956/
1,u19725,19/12/14 13:52,19/12/14 13:52,A00002/B00005/C00067/D02026/
2,u11795,01/12/14 10:44,01/12/14 10:44,A00002/B00002/C00004/D12538/
3,u22639,08/12/14 20:19,08/12/14 20:22,A00002/B00003/C00079/D22781/;A00002/B00003/C00...
4,u18034,15/12/14 19:33,15/12/14 19:33,A00002/B00001/C00010/D23419/


In [62]:
data.head()

,session_id,startTime,endTime,ProductList
0,u16159,15/12/14 18:11,15/12/14 18:12,A00002/B00003/C00006/D28435/;A00002/B00003/C00...
1,u10253,16/12/14 14:35,16/12/14 14:41,A00001/B00009/C00031/D29404/;A00001/B00009/C00...
2,u19037,01/12/14 15:58,01/12/14 15:58,A00002/B00001/C00020/D16944/
3,u14556,23/11/14 2:57,23/11/14 3:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00...
4,u24295,17/12/14 16:44,17/12/14 16:46,A00001/B00001/C00012/D30805/;A00001/B00001/C00...


In [0]:
data['train']=1   # 1 for training data

In [64]:
data

,session_id,startTime,endTime,ProductList,train
0,u16159,15/12/14 18:11,15/12/14 18:12,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,1
1,u10253,16/12/14 14:35,16/12/14 14:41,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,1
2,u19037,01/12/14 15:58,01/12/14 15:58,A00002/B00001/C00020/D16944/,1
3,u14556,23/11/14 2:57,23/11/14 3:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,1
4,u24295,17/12/14 16:44,17/12/14 16:46,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,1
...,...,...,...,...,...
10495,u15442,18/11/14 7:39,18/11/14 7:42,A00002/B00002/C00007/D06407/;A00002/B00002/C00...,1
10496,u17986,25/11/14 15:16,25/11/14 15:16,A00006/B00030/C00334/D11660/,1
10497,u22508,09/12/14 10:11,09/12/14 10:11,A00002/B00002/C00007/D18028/,1
10498,u17087,22/11/14 11:27,22/11/14 11:27,A00003/B00012/C00131/D09453/;A00003/B00012/C00...,1


In [0]:
test['train']=0  # 0 for test data it will use further to divide it into train in test

In [66]:
test

,session_id,startTime,endTime,ProductList,train
0,u12112,08/12/14 13:36,08/12/14 13:36,A00002/B00003/C00006/D19956/,0
1,u19725,19/12/14 13:52,19/12/14 13:52,A00002/B00005/C00067/D02026/,0
2,u11795,01/12/14 10:44,01/12/14 10:44,A00002/B00002/C00004/D12538/,0
3,u22639,08/12/14 20:19,08/12/14 20:22,A00002/B00003/C00079/D22781/;A00002/B00003/C00...,0
4,u18034,15/12/14 19:33,15/12/14 19:33,A00002/B00001/C00010/D23419/,0
...,...,...,...,...,...
4495,u23966,15/12/14 21:11,15/12/14 21:12,A00001/B00031/C00044/D03435/;A00001/B00031/C00...,0
4496,u20527,02/12/14 7:53,02/12/14 7:54,A00002/B00002/C00004/D17474/;A00002/B00002/C00...,0
4497,u13253,15/11/14 21:38,15/11/14 21:46,A00002/B00007/C00098/D03350/;A00002/B00007/C00...,0
4498,u17094,17/12/14 8:51,17/12/14 8:51,A00001/B00031/C00044/D09604/,0


In [0]:
combined=pd.concat([data,test],ignore_index=True)  #combined test nd train to avoid number of features mismatch error

In [68]:
combined.train.value_counts()   # 1 means train data and 0 means test data

1    10500
0     4500
Name: train, dtype: int64

In [69]:
combined

,session_id,startTime,endTime,ProductList,train
0,u16159,15/12/14 18:11,15/12/14 18:12,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,1
1,u10253,16/12/14 14:35,16/12/14 14:41,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,1
2,u19037,01/12/14 15:58,01/12/14 15:58,A00002/B00001/C00020/D16944/,1
3,u14556,23/11/14 2:57,23/11/14 3:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,1
4,u24295,17/12/14 16:44,17/12/14 16:46,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,1
...,...,...,...,...,...
14995,u23966,15/12/14 21:11,15/12/14 21:12,A00001/B00031/C00044/D03435/;A00001/B00031/C00...,0
14996,u20527,02/12/14 7:53,02/12/14 7:54,A00002/B00002/C00004/D17474/;A00002/B00002/C00...,0
14997,u13253,15/11/14 21:38,15/11/14 21:46,A00002/B00007/C00098/D03350/;A00002/B00007/C00...,0
14998,u17094,17/12/14 8:51,17/12/14 8:51,A00001/B00031/C00044/D09604/,0


In [70]:
x=combined['ProductList'][0]
from collections import Counter 
if ";"  in x:
    prd_lst = x.split(";")
    print(prd_lst)
    
    count_item = len(prd_lst)
    #print(count_item)
    first_lv1 = prd_lst[0].split("/")[0]
    first_lv2 = prd_lst[0].split("/")[1]
    #print(first_lv1)
    #print(first_lv2)
    lv1_lst =[]
    lv2_lst =[]
    for item in prd_lst:
        lv1_lst.append(item.split("/")[0])
        lv2_lst.append(item.split('/')[1])
    print(lv1_lst)
    print(lv2_lst)
    unique_lv1 = len(set(lv1_lst))
    unique_lv2 = len(set(lv2_lst))
    print(unique_lv1)
    most_freq_lv1 =  max(lv1_lst, key=Counter(lv1_lst).get)
    print(most_freq_lv1)
else:
    lv_lst = str.split("/")
    first_lv1 = lv_lst[0]
    first_lv2 = lv_lst[1]
    count_item = 1
    unique_lv1 = 1
    unique_lv2 = 1
    most_freq_lv1 = first_lv1

['A00002/B00003/C00006/D28435/', 'A00002/B00003/C00006/D02554/', 'A00002/B00003/C00006/D28436/', 'A00002/B00003/C00006/D28437/']
['A00002', 'A00002', 'A00002', 'A00002']
['B00003', 'B00003', 'B00003', 'B00003']
1
A00002


## function to seperate categories

In [0]:
def extract_product(str):
    if ";"  in str:
        prd_lst = str.split(";")
        count_item = len(prd_lst)
        first_lv1 = prd_lst[0].split("/")[0]
        first_lv2 = prd_lst[0].split("/")[1]
        lv1_lst =[]
        lv2_lst =[]
        for item in prd_lst:
            lv1_lst.append(item.split("/")[0])
            lv2_lst.append(item.split("/")[1])
        unique_lv1 = len(set(lv1_lst))
        unique_lv2 = len(set(lv2_lst))
        most_freq_lv1 =  max(lv1_lst, key=Counter(lv1_lst).get)        
    else:
        lv_lst = str.split("/")
        first_lv1 = lv_lst[0]
        first_lv2 = lv_lst[1]
        count_item = 1
        unique_lv1 = 1
        unique_lv2 = 1
        most_freq_lv1 = first_lv1
    return (count_item,first_lv1,first_lv2,unique_lv1,unique_lv2,most_freq_lv1)    


In [0]:
#Num of product = no of products seperate by ';'
#FirstA= First category seperated by '/'
#FirstB= first subcategory seperated by '/'
#UniqueA= unique FirstA seperated by ';'
#uniqueB= unique FirstB seperated by ';'
#MostA= maximun occured category

In [0]:
new_col = ('Num_of_Product','FirstA','FirstB','UniqueA','UniqueB','MostA')      
new_col_lst = combined['ProductList'].apply(lambda x: extract_product(x)) #apply on ProductList feature

In [74]:
new_col_df = pd.DataFrame(new_col_lst.tolist(),columns =new_col)
new_col_df
#dataframe of ProductList which is seperated in Categories

,Num_of_Product,FirstA,FirstB,UniqueA,UniqueB,MostA
0,4,A00002,B00003,1,1,A00002
1,7,A00001,B00009,1,1,A00001
2,1,A00002,B00001,1,1,A00002
3,3,A00002,B00004,1,1,A00002
4,2,A00001,B00001,1,1,A00001
...,...,...,...,...,...,...
14995,3,A00001,B00031,1,1,A00001
14996,2,A00002,B00002,1,1,A00002
14997,9,A00002,B00007,1,1,A00002
14998,1,A00001,B00031,1,1,A00001


In [0]:
combined= pd.concat([combined, new_col_df],axis=1) #combine with original dataframe

In [76]:
combined

,session_id,startTime,endTime,ProductList,train,Num_of_Product,FirstA,FirstB,UniqueA,UniqueB,MostA
0,u16159,15/12/14 18:11,15/12/14 18:12,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,1,4,A00002,B00003,1,1,A00002
1,u10253,16/12/14 14:35,16/12/14 14:41,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,1,7,A00001,B00009,1,1,A00001
2,u19037,01/12/14 15:58,01/12/14 15:58,A00002/B00001/C00020/D16944/,1,1,A00002,B00001,1,1,A00002
3,u14556,23/11/14 2:57,23/11/14 3:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,1,3,A00002,B00004,1,1,A00002
4,u24295,17/12/14 16:44,17/12/14 16:46,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,1,2,A00001,B00001,1,1,A00001
...,...,...,...,...,...,...,...,...,...,...,...
14995,u23966,15/12/14 21:11,15/12/14 21:12,A00001/B00031/C00044/D03435/;A00001/B00031/C00...,0,3,A00001,B00031,1,1,A00001
14996,u20527,02/12/14 7:53,02/12/14 7:54,A00002/B00002/C00004/D17474/;A00002/B00002/C00...,0,2,A00002,B00002,1,1,A00002
14997,u13253,15/11/14 21:38,15/11/14 21:46,A00002/B00007/C00098/D03350/;A00002/B00007/C00...,0,9,A00002,B00007,1,1,A00002
14998,u17094,17/12/14 8:51,17/12/14 8:51,A00001/B00031/C00044/D09604/,0,1,A00001,B00031,1,1,A00001


In [0]:
df=pd.get_dummies(combined[['FirstA','FirstB','MostA']]) #one hot encoding on categorical features

In [78]:
df

,FirstA_A00001,FirstA_A00002,FirstA_A00003,FirstA_A00004,FirstA_A00005,FirstA_A00006,FirstA_A00007,FirstA_A00008,FirstA_A00009,FirstA_A00010,FirstA_A00011,FirstB_B00001,FirstB_B00002,FirstB_B00003,FirstB_B00004,FirstB_B00005,FirstB_B00006,FirstB_B00007,FirstB_B00008,FirstB_B00009,FirstB_B00010,FirstB_B00011,FirstB_B00012,FirstB_B00013,FirstB_B00014,FirstB_B00015,FirstB_B00016,FirstB_B00017,FirstB_B00018,FirstB_B00019,FirstB_B00020,FirstB_B00021,FirstB_B00022,FirstB_B00023,FirstB_B00024,FirstB_B00025,FirstB_B00026,FirstB_B00027,FirstB_B00028,FirstB_B00029,...,FirstB_B00056,FirstB_B00057,FirstB_B00058,FirstB_B00059,FirstB_B00060,FirstB_B00061,FirstB_B00062,FirstB_B00063,FirstB_B00064,FirstB_B00065,FirstB_B00066,FirstB_B00067,FirstB_B00069,FirstB_B00070,FirstB_B00071,FirstB_B00072,FirstB_B00073,FirstB_B00074,FirstB_B00075,FirstB_B00076,FirstB_B00077,FirstB_B00078,FirstB_B00079,FirstB_B00080,FirstB_B00082,FirstB_B00083,FirstB_B00085,FirstB_B00088,FirstB_B00091,MostA_A00001,MostA_A00002,MostA_A00003,MostA_A00004,MostA_A00005,MostA_A00006,MostA_A00007,MostA_A00008,MostA_A00009,MostA_A00010,MostA_A00011
0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
14996,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
14997,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
14998,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [0]:
combined=pd.concat([combined,df],axis=1) #concat one hot encoded dataframe with original dataframe 

In [0]:
#data = pd.concat([data, new_col_df], axis=1)

In [81]:
combined

,session_id,startTime,endTime,ProductList,train,Num_of_Product,FirstA,FirstB,UniqueA,UniqueB,MostA,FirstA_A00001,FirstA_A00002,FirstA_A00003,FirstA_A00004,FirstA_A00005,FirstA_A00006,FirstA_A00007,FirstA_A00008,FirstA_A00009,FirstA_A00010,FirstA_A00011,FirstB_B00001,FirstB_B00002,FirstB_B00003,FirstB_B00004,FirstB_B00005,FirstB_B00006,FirstB_B00007,FirstB_B00008,FirstB_B00009,FirstB_B00010,FirstB_B00011,FirstB_B00012,FirstB_B00013,FirstB_B00014,FirstB_B00015,FirstB_B00016,FirstB_B00017,FirstB_B00018,...,FirstB_B00056,FirstB_B00057,FirstB_B00058,FirstB_B00059,FirstB_B00060,FirstB_B00061,FirstB_B00062,FirstB_B00063,FirstB_B00064,FirstB_B00065,FirstB_B00066,FirstB_B00067,FirstB_B00069,FirstB_B00070,FirstB_B00071,FirstB_B00072,FirstB_B00073,FirstB_B00074,FirstB_B00075,FirstB_B00076,FirstB_B00077,FirstB_B00078,FirstB_B00079,FirstB_B00080,FirstB_B00082,FirstB_B00083,FirstB_B00085,FirstB_B00088,FirstB_B00091,MostA_A00001,MostA_A00002,MostA_A00003,MostA_A00004,MostA_A00005,MostA_A00006,MostA_A00007,MostA_A00008,MostA_A00009,MostA_A00010,MostA_A00011
0,u16159,15/12/14 18:11,15/12/14 18:12,A00002/B00003/C00006/D28435/;A00002/B00003/C00...,1,4,A00002,B00003,1,1,A00002,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,u10253,16/12/14 14:35,16/12/14 14:41,A00001/B00009/C00031/D29404/;A00001/B00009/C00...,1,7,A00001,B00009,1,1,A00001,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,u19037,01/12/14 15:58,01/12/14 15:58,A00002/B00001/C00020/D16944/,1,1,A00002,B00001,1,1,A00002,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,u14556,23/11/14 2:57,23/11/14 3:00,A00002/B00004/C00018/D10284/;A00002/B00004/C00...,1,3,A00002,B00004,1,1,A00002,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,u24295,17/12/14 16:44,17/12/14 16:46,A00001/B00001/C00012/D30805/;A00001/B00001/C00...,1,2,A00001,B00001,1,1,A00001,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,u23966,15/12/14 21:11,15/12/14 21:12,A00001/B00031/C00044/D03435/;A00001/B00031/C00...,0,3,A00001,B00031,1,1,A00001,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
14996,u20527,02/12/14 7:53,02/12/14 7:54,A00002/B00002/C00004/D17474/;A00002/B00002/C00...,0,2,A00002,B00002,1,1,A00002,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
14997,u13253,15/11/14 21:38,15/11/14 21:46,A00002/B00007/C00098/D03350/;A00002/B00007/C00...,0,9,A00002,B00007,1,1,A00002,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
14998,u17094,17/12/14 8:51,17/12/14 8:51,A00001/B00031/C00044/D09604/,0,1,A00001,B00031,1,1,A00001,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [0]:
combined['startTime'] = pd.to_datetime(combined['startTime']) #converting into DateTime from object type
combined['endTime'] = pd.to_datetime(combined['endTime'])
combined['duration'] = combined['endTime'] - combined['startTime'] #taking time difference
combined['duration'] = combined['duration'].astype('timedelta64[m]') #duration in minutes
combined['weekday'] = combined['startTime'].dt.dayofweek             #day of week
combined['hour_24h'] = combined['startTime'].dt.hour                 #duration hour

In [0]:
drop_lst = ['session_id','startTime', 'endTime', 'ProductList','FirstA','FirstB','MostA']
# above are fetures to be drop          
combined = combined.drop(drop_lst,axis =1 )


In [84]:
combined # whole dataframe with numerical features only

,train,Num_of_Product,UniqueA,UniqueB,FirstA_A00001,FirstA_A00002,FirstA_A00003,FirstA_A00004,FirstA_A00005,FirstA_A00006,FirstA_A00007,FirstA_A00008,FirstA_A00009,FirstA_A00010,FirstA_A00011,FirstB_B00001,FirstB_B00002,FirstB_B00003,FirstB_B00004,FirstB_B00005,FirstB_B00006,FirstB_B00007,FirstB_B00008,FirstB_B00009,FirstB_B00010,FirstB_B00011,FirstB_B00012,FirstB_B00013,FirstB_B00014,FirstB_B00015,FirstB_B00016,FirstB_B00017,FirstB_B00018,FirstB_B00019,FirstB_B00020,FirstB_B00021,FirstB_B00022,FirstB_B00023,FirstB_B00024,FirstB_B00025,...,FirstB_B00059,FirstB_B00060,FirstB_B00061,FirstB_B00062,FirstB_B00063,FirstB_B00064,FirstB_B00065,FirstB_B00066,FirstB_B00067,FirstB_B00069,FirstB_B00070,FirstB_B00071,FirstB_B00072,FirstB_B00073,FirstB_B00074,FirstB_B00075,FirstB_B00076,FirstB_B00077,FirstB_B00078,FirstB_B00079,FirstB_B00080,FirstB_B00082,FirstB_B00083,FirstB_B00085,FirstB_B00088,FirstB_B00091,MostA_A00001,MostA_A00002,MostA_A00003,MostA_A00004,MostA_A00005,MostA_A00006,MostA_A00007,MostA_A00008,MostA_A00009,MostA_A00010,MostA_A00011,duration,weekday,hour_24h
0,1,4,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1.0,0,18
1,1,7,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6.0,1,14
2,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,6,15
3,1,3,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3.0,6,2
4,1,2,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2.0,2,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,0,3,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1.0,0,21
14996,0,2,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1.0,2,7
14997,0,9,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,8.0,5,21
14998,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,2,8


In [0]:
train_df=combined[combined['train'] == 1] # seperating train data which we have combined earlier with test.csv
test_df=combined[combined['train'] == 0]  #seperating test data using train==0.

In [86]:
train_df  # final train data

,train,Num_of_Product,UniqueA,UniqueB,FirstA_A00001,FirstA_A00002,FirstA_A00003,FirstA_A00004,FirstA_A00005,FirstA_A00006,FirstA_A00007,FirstA_A00008,FirstA_A00009,FirstA_A00010,FirstA_A00011,FirstB_B00001,FirstB_B00002,FirstB_B00003,FirstB_B00004,FirstB_B00005,FirstB_B00006,FirstB_B00007,FirstB_B00008,FirstB_B00009,FirstB_B00010,FirstB_B00011,FirstB_B00012,FirstB_B00013,FirstB_B00014,FirstB_B00015,FirstB_B00016,FirstB_B00017,FirstB_B00018,FirstB_B00019,FirstB_B00020,FirstB_B00021,FirstB_B00022,FirstB_B00023,FirstB_B00024,FirstB_B00025,...,FirstB_B00059,FirstB_B00060,FirstB_B00061,FirstB_B00062,FirstB_B00063,FirstB_B00064,FirstB_B00065,FirstB_B00066,FirstB_B00067,FirstB_B00069,FirstB_B00070,FirstB_B00071,FirstB_B00072,FirstB_B00073,FirstB_B00074,FirstB_B00075,FirstB_B00076,FirstB_B00077,FirstB_B00078,FirstB_B00079,FirstB_B00080,FirstB_B00082,FirstB_B00083,FirstB_B00085,FirstB_B00088,FirstB_B00091,MostA_A00001,MostA_A00002,MostA_A00003,MostA_A00004,MostA_A00005,MostA_A00006,MostA_A00007,MostA_A00008,MostA_A00009,MostA_A00010,MostA_A00011,duration,weekday,hour_24h
0,1,4,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1.0,0,18
1,1,7,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6.0,1,14
2,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,6,15
3,1,3,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3.0,6,2
4,1,2,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2.0,2,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,1,2,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3.0,1,7
10496,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0.0,1,15
10497,1,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,4,10
10498,1,2,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0.0,5,11


In [0]:
train_df.drop(['train'],axis=1,inplace=True) # drop train feature which we have added to identify train and test
test_df.drop(['train'],axis=1,inplace=True)

In [88]:
train_df.head()


,Num_of_Product,UniqueA,UniqueB,FirstA_A00001,FirstA_A00002,FirstA_A00003,FirstA_A00004,FirstA_A00005,FirstA_A00006,FirstA_A00007,FirstA_A00008,FirstA_A00009,FirstA_A00010,FirstA_A00011,FirstB_B00001,FirstB_B00002,FirstB_B00003,FirstB_B00004,FirstB_B00005,FirstB_B00006,FirstB_B00007,FirstB_B00008,FirstB_B00009,FirstB_B00010,FirstB_B00011,FirstB_B00012,FirstB_B00013,FirstB_B00014,FirstB_B00015,FirstB_B00016,FirstB_B00017,FirstB_B00018,FirstB_B00019,FirstB_B00020,FirstB_B00021,FirstB_B00022,FirstB_B00023,FirstB_B00024,FirstB_B00025,FirstB_B00026,...,FirstB_B00059,FirstB_B00060,FirstB_B00061,FirstB_B00062,FirstB_B00063,FirstB_B00064,FirstB_B00065,FirstB_B00066,FirstB_B00067,FirstB_B00069,FirstB_B00070,FirstB_B00071,FirstB_B00072,FirstB_B00073,FirstB_B00074,FirstB_B00075,FirstB_B00076,FirstB_B00077,FirstB_B00078,FirstB_B00079,FirstB_B00080,FirstB_B00082,FirstB_B00083,FirstB_B00085,FirstB_B00088,FirstB_B00091,MostA_A00001,MostA_A00002,MostA_A00003,MostA_A00004,MostA_A00005,MostA_A00006,MostA_A00007,MostA_A00008,MostA_A00009,MostA_A00010,MostA_A00011,duration,weekday,hour_24h
0,4,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1.0,0,18
1,7,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6.0,1,14
2,1,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,6,15
3,3,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3.0,6,2
4,2,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2.0,2,16


In [89]:
test_df.head()

,Num_of_Product,UniqueA,UniqueB,FirstA_A00001,FirstA_A00002,FirstA_A00003,FirstA_A00004,FirstA_A00005,FirstA_A00006,FirstA_A00007,FirstA_A00008,FirstA_A00009,FirstA_A00010,FirstA_A00011,FirstB_B00001,FirstB_B00002,FirstB_B00003,FirstB_B00004,FirstB_B00005,FirstB_B00006,FirstB_B00007,FirstB_B00008,FirstB_B00009,FirstB_B00010,FirstB_B00011,FirstB_B00012,FirstB_B00013,FirstB_B00014,FirstB_B00015,FirstB_B00016,FirstB_B00017,FirstB_B00018,FirstB_B00019,FirstB_B00020,FirstB_B00021,FirstB_B00022,FirstB_B00023,FirstB_B00024,FirstB_B00025,FirstB_B00026,...,FirstB_B00059,FirstB_B00060,FirstB_B00061,FirstB_B00062,FirstB_B00063,FirstB_B00064,FirstB_B00065,FirstB_B00066,FirstB_B00067,FirstB_B00069,FirstB_B00070,FirstB_B00071,FirstB_B00072,FirstB_B00073,FirstB_B00074,FirstB_B00075,FirstB_B00076,FirstB_B00077,FirstB_B00078,FirstB_B00079,FirstB_B00080,FirstB_B00082,FirstB_B00083,FirstB_B00085,FirstB_B00088,FirstB_B00091,MostA_A00001,MostA_A00002,MostA_A00003,MostA_A00004,MostA_A00005,MostA_A00006,MostA_A00007,MostA_A00008,MostA_A00009,MostA_A00010,MostA_A00011,duration,weekday,hour_24h
10500,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,1,13
10501,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,4,13
10502,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,6,10
10503,4,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3.0,1,20
10504,1,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0,19


### save train and test preprocessed data into pickle so whenever we want to apply any algorithm we can directly load preprocessed files. 

In [0]:

with open('test.pickle', 'wb') as f:
    pickle.dump(test_df, f)

In [0]:
with open('/content/test.pickle', 'rb') as f:
    test_df = pickle.load(f)

In [92]:
test_df.head()

,Num_of_Product,UniqueA,UniqueB,FirstA_A00001,FirstA_A00002,FirstA_A00003,FirstA_A00004,FirstA_A00005,FirstA_A00006,FirstA_A00007,FirstA_A00008,FirstA_A00009,FirstA_A00010,FirstA_A00011,FirstB_B00001,FirstB_B00002,FirstB_B00003,FirstB_B00004,FirstB_B00005,FirstB_B00006,FirstB_B00007,FirstB_B00008,FirstB_B00009,FirstB_B00010,FirstB_B00011,FirstB_B00012,FirstB_B00013,FirstB_B00014,FirstB_B00015,FirstB_B00016,FirstB_B00017,FirstB_B00018,FirstB_B00019,FirstB_B00020,FirstB_B00021,FirstB_B00022,FirstB_B00023,FirstB_B00024,FirstB_B00025,FirstB_B00026,...,FirstB_B00059,FirstB_B00060,FirstB_B00061,FirstB_B00062,FirstB_B00063,FirstB_B00064,FirstB_B00065,FirstB_B00066,FirstB_B00067,FirstB_B00069,FirstB_B00070,FirstB_B00071,FirstB_B00072,FirstB_B00073,FirstB_B00074,FirstB_B00075,FirstB_B00076,FirstB_B00077,FirstB_B00078,FirstB_B00079,FirstB_B00080,FirstB_B00082,FirstB_B00083,FirstB_B00085,FirstB_B00088,FirstB_B00091,MostA_A00001,MostA_A00002,MostA_A00003,MostA_A00004,MostA_A00005,MostA_A00006,MostA_A00007,MostA_A00008,MostA_A00009,MostA_A00010,MostA_A00011,duration,weekday,hour_24h
10500,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,1,13
10501,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,4,13
10502,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,6,10
10503,4,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3.0,1,20
10504,1,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0,19


# Train Data

In [0]:
# Devide data set into train set , test set
X = train_df   # preprocessed training data which is availabe in train.csv
y= Y['gender'].map({'female':1,'male':0})  #dependent feature which want to predict on.


In [0]:
import pickle
with open('train.pickle', 'wb') as f:
    pickle.dump([X, y], f)

In [0]:
with open('/content/drive/My Drive/Colab Notebooks/extra/train.pickle', 'rb') as f:
    X, y = pickle.load(f)

In [48]:
X.shape

(10500, 112)

In [49]:
X.corr()

,Num_of_Product,UniqueA,UniqueB,FirstA_A00001,FirstA_A00002,FirstA_A00003,FirstA_A00004,FirstA_A00005,FirstA_A00006,FirstA_A00007,FirstA_A00008,FirstA_A00009,FirstA_A00010,FirstA_A00011,FirstB_B00001,FirstB_B00002,FirstB_B00003,FirstB_B00004,FirstB_B00005,FirstB_B00006,FirstB_B00007,FirstB_B00008,FirstB_B00009,FirstB_B00010,FirstB_B00011,FirstB_B00012,FirstB_B00013,FirstB_B00014,FirstB_B00015,FirstB_B00016,FirstB_B00017,FirstB_B00018,FirstB_B00019,FirstB_B00020,FirstB_B00021,FirstB_B00022,FirstB_B00023,FirstB_B00024,FirstB_B00025,FirstB_B00026,...,FirstB_B00059,FirstB_B00060,FirstB_B00061,FirstB_B00062,FirstB_B00063,FirstB_B00064,FirstB_B00065,FirstB_B00066,FirstB_B00067,FirstB_B00069,FirstB_B00070,FirstB_B00071,FirstB_B00072,FirstB_B00073,FirstB_B00074,FirstB_B00075,FirstB_B00076,FirstB_B00077,FirstB_B00078,FirstB_B00079,FirstB_B00080,FirstB_B00082,FirstB_B00083,FirstB_B00085,FirstB_B00088,FirstB_B00091,MostA_A00001,MostA_A00002,MostA_A00003,MostA_A00004,MostA_A00005,MostA_A00006,MostA_A00007,MostA_A00008,MostA_A00009,MostA_A00010,MostA_A00011,duration,weekday,hour_24h
Num_of_Product,1.000000,0.187165,0.397291,-0.015260,0.003305,0.030268,-0.015808,-0.013857,-0.023788,0.001182,-0.006952,-0.007962,-0.018003,0.014807,-0.010781,0.021492,0.001361,-0.022186,0.007820,-0.008433,0.003570,-0.005665,-0.011620,0.010166,0.017818,0.029024,-0.017558,-0.006066,-0.010535,-0.015788,0.007698,-0.012956,-0.019601,0.016764,0.012134,0.009227,-0.006795,-0.009233,0.015479,-0.016972,...,0.010373,-0.006795,-0.007250,-0.001810,-0.008144,-0.007250,0.011005,-0.011466,-0.005127,-0.011291,0.012370,-0.006443,-0.000905,-0.001280,0.007538,-0.005127,0.007676,0.004936,-0.005127,-0.001280,-0.000905,-0.005127,-0.005127,-0.005127,NaN,NaN,-0.021997,0.002374,0.038412,-0.015787,-0.013021,-0.020065,-0.006221,-0.009684,-0.011252,-0.017327,0.015129,0.004964,0.016489,0.014780
UniqueA,0.187165,1.000000,0.535106,0.054096,-0.093665,0.014723,0.031082,0.041991,0.032509,0.009356,0.022211,0.027600,0.037392,0.035411,-0.001263,-0.059028,-0.014715,0.018395,-0.017242,-0.007750,-0.022774,-0.004698,0.025591,-0.009004,0.011383,0.009883,0.017199,0.022367,0.026860,-0.015167,0.018619,0.039421,0.004040,-0.000622,0.019933,-0.002823,0.035598,0.007435,0.020141,-0.001324,...,0.013357,0.021516,-0.002712,0.020544,-0.003836,-0.002712,0.023050,-0.004289,-0.001918,0.031934,0.029060,-0.003322,0.046836,0.031763,0.046836,-0.001918,-0.002712,0.060730,-0.001918,-0.002712,0.046836,-0.001918,-0.001918,-0.001918,NaN,NaN,0.033744,-0.088885,0.027570,0.027193,0.048426,0.045591,-0.009599,0.001853,0.018929,0.043841,0.041696,-0.027477,-0.008920,0.014395
UniqueB,0.397291,0.535106,1.000000,0.023477,-0.011241,-0.017889,-0.007413,0.021896,0.000303,-0.003858,0.012761,0.004085,0.003645,0.005275,-0.005312,-0.043663,0.013465,-0.003778,0.032439,0.022452,0.005276,-0.008093,0.030727,-0.006085,0.027771,-0.017492,-0.001463,-0.008703,-0.004022,-0.002222,0.009233,0.013163,0.005782,-0.014529,0.001015,-0.018773,0.007681,-0.005072,0.006463,-0.004078,...,0.024645,0.007681,-0.004672,0.004433,-0.006607,-0.004672,0.022338,-0.007387,-0.003303,0.011249,0.021886,-0.005722,0.018774,0.026551,0.018774,-0.003303,-0.004672,0.024843,-0.003303,0.010940,0.018774,-0.003303,-0.003303,-0.003303,NaN,NaN,0.014653,-0.007471,-0.013154,-0.010927,0.024451,0.001470,-0.016535,0.004137,0.000476,0.012806,0.007883,-0.003586,-0.001280,0.010148
FirstA_A00001,-0.015260,0.054096,0.023477,1.000000,-0.562458,-0.177063,-0.048998,-0.058883,-0.042565,-0.021000,-0.019314,-0.021401,-0.027960,-0.028861,0.185047,-0.230796,-0.162382,0.117745,-0.068995,-0.050913,-0.108187,-0.010079,0.447676,-0.019314,-0.009412,-0.119152,-0.027029,-0.039628,0.378046,-0.061230,-0.055192,-0.016974,-0.031953,-0.042362,-0.044743,-0.072904,-0.014257,-0.014257,-0.014840,-0.030864,...,-0.010887,-0.014257,-0.005818,-0.008228,-0.008228,-0.005818,-0.013650,-0.009200,-0.004114,-0.015401,-0.011639,-0.007126,-0.004114,-0.005818,-0.004114,-0.004114,-0.005818,-0.014840,-0.004114,-0.005818,-0.004114,-0.004

In [93]:
X_train, X_cv, Y_train, Y_cv = train_test_split(X, y, test_size=0.25)
print(X.shape)           # shape of whole train data
print(X_train.shape)     # shape of X_train after splitting.  75% train data
print(X_cv.shape)        # shaape of X_cv after splitting.    25% cross validation data   

(10500, 112)
(7875, 112)
(2625, 112)


In [94]:
type(X_train)

pandas.core.frame.DataFrame

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import seaborn as sns

## Hyperparameter Tunning

In [0]:
DT = DecisionTreeClassifier()
parameters = {'max_depth':[1,5,10,50,100,500], 'min_samples_split': [2,5,10,100,500],'min_samples_leaf': [1,2,5]}
classifier = GridSearchCV(DT, parameters, cv=3,return_train_score=True)
classifier.fit(X_train, Y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_depth': [1, 5, 10, 50, 100, 500],
              

In [0]:
print('Best score: %0.3f' % classifier.best_score_)

Best score: 0.869


In [0]:
print('Best tuned parameters:')
print(classifier.best_params_)

Best tuned parameters:
{'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 2}


In [0]:
predictions = classifier.predict(X_cv)
print(classification_report(Y_cv, predictions))
print("======>Confusion matrix : ")
print(confusion_matrix(Y_cv, classifier.predict(X_cv)))


              precision    recall  f1-score   support

           0       0.79      0.60      0.68       583
           1       0.89      0.95      0.92      2042

    accuracy                           0.88      2625
   macro avg       0.84      0.78      0.80      2625
weighted avg       0.87      0.88      0.87      2625

======>Confusion matrix : 
[[ 352  231]
 [  94 1948]]


### Train model using Best hyper parameters

In [0]:
dt=DecisionTreeClassifier(max_depth= 5, min_samples_leaf= 5, min_samples_split= 2)
dt.fit(X_train,Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

## Top 10 important features by using DT

In [0]:
for importance, name in sorted(zip(dt.feature_importances_, X_train.columns),reverse=True)[:10]:
    print (name, importance)

MostA_A00001 0.8411703692874131
MostA_A00004 0.06387348945572541
weekday 0.020797840147794965
MostA_A00010 0.018741832325196287
hour_24h 0.016082754470410944
Num_of_Product 0.011217474516300518
FirstB_B00026 0.005744011916770508
FirstB_B00015 0.005285437406950658
FirstB_B00002 0.004967438458881844
duration 0.00415808458332493


# Checking CV score using KFold on Different Algorithms

*   **Bagging Algorithms**

## Bagged Decision Trees

In [0]:
# Bagged Decision Trees for Classification
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [0]:
kfold = model_selection.KFold(n_splits=10, random_state=7)
cart = DecisionTreeClassifier(max_depth= 5, min_samples_leaf= 5, min_samples_split= 2)
num_trees = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=7)
results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold)
print(results.mean())

0.8708560426731339


## RandomForest

In [0]:
seed = 7
num_trees = 100
max_features = 3
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold)
print(results.mean())

0.8523158689104033


# **Boosting Algorithms**
### Adaboost

In [0]:
from sklearn.ensemble import AdaBoostClassifier
seed = 7
num_trees = 30
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold)
print(results.mean())

0.8698390404994873


## Stochastic Gradient Boosting

In [0]:
from sklearn.ensemble import GradientBoostingClassifier
seed = 7
num_trees = 100
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = GradientBoostingClassifier(n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold)
print(results.mean())

0.8718715935990298


# Voting Ensemble

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

In [0]:
seed = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
# create the sub models
estimators = []
model1 = LogisticRegression()
estimators.append(('logistic', model1))
model2 = DecisionTreeClassifier()
estimators.append(('cart', model2))
model3 = SVC()
estimators.append(('svm', model3))
# create the ensemble model
ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, X_train, Y_train, cv=kfold)
print(results.mean())

0.8416583891794968


# Feature importance using RandomForest

In [0]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=2, random_state=0)
rf.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:
print('CV_accuracy',rf.score(X_cv,Y_cv))


CV_accuracy 0.7939047619047619


In [0]:
predictions = rf.predict(X_cv)
print(classification_report(Y_cv, predictions))
print("======>Confusion matrix : ")
print(confusion_matrix(Y_cv, rf.predict(X_cv)))

              precision    recall  f1-score   support

           0       0.83      0.09      0.16       583
           1       0.79      0.99      0.88      2042

    accuracy                           0.79      2625
   macro avg       0.81      0.54      0.52      2625
weighted avg       0.80      0.79      0.72      2625

======>Confusion matrix : 
[[  53  530]
 [  11 2031]]


In [0]:
for importance, name in sorted(zip(rf.feature_importances_, X_train.columns),reverse=True)[:20]:
    print (name, importance)

FirstA_A00001 0.15462495028663556
MostA_A00001 0.11402214741865242
FirstB_B00015 0.10676463911722034
FirstA_A00002 0.10067762725363544
MostA_A00002 0.07382456484632777
FirstB_B00009 0.07231917592826428
FirstB_B00031 0.055111501467106326
FirstB_B00002 0.04071773774262543
FirstA_A00004 0.03899298345858313
FirstB_B00001 0.03170659538287535
FirstA_A00003 0.028164339951730302
FirstB_B00003 0.027630044378275133
FirstB_B00037 0.026474868582674355
MostA_A00004 0.025758244111079354
FirstB_B00012 0.016284351259709163
FirstB_B00014 0.012611951733977247
FirstB_B00027 0.01098656905494961
FirstB_B00007 0.010771556843938239
hour_24h 0.007622158047804386
FirstB_B00004 0.007538864374257738


#XGBoost

In [137]:
#import XGBoost classifier and accuracy
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

#instantiate model and train
model = XGBClassifier(learning_rate = 0.1, n_estimators=500, max_depth=5,subsample=1,colsample_bytree=0.2,colsample_bynode=0.1,colsample_bylevel=0.1)
model.fit(X_train, Y_train)

# make predictions for test set
y_pred = model.predict(X_cv)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(Y_cv, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 86.86%


##Feature Importance Using XGBoost

In [138]:
for importance, name in sorted(zip(model.feature_importances_, X_train.columns),reverse=True)[:20]:
    print (name, importance)

FirstA_A00001 0.33715606
MostA_A00001 0.20677769
MostA_A00002 0.13827604
FirstA_A00002 0.0499376
FirstB_B00002 0.025219852
FirstB_B00015 0.024963195
FirstA_A00003 0.021560978
FirstB_B00009 0.021530584
FirstB_B00031 0.020075526
MostA_A00003 0.01735791
FirstB_B00001 0.014847342
MostA_A00004 0.012853919
FirstA_A00004 0.01052568
FirstB_B00012 0.0076723
MostA_A00010 0.007506467
hour_24h 0.0069704996
FirstB_B00003 0.0062776217
FirstB_B00007 0.0057425085
FirstB_B00026 0.005104874
FirstB_B00014 0.004785766


*   Top 50 Features

In [0]:
#Top 50 Features
thresholds = sorted(model.feature_importances_,reverse=True)
tr=thresholds[:50]

In [0]:
from sklearn.feature_selection import SelectFromModel

In [135]:
for thresh in tr:
	# select features using threshold
	selection = SelectFromModel(model, threshold=thresh, prefit=True)
	select_X_train = selection.transform(X_train)
	# train model
	selection_model = XGBClassifier()
	selection_model.fit(select_X_train, Y_train)
	# eval model
	select_X_test = selection.transform(X_cv)
	y_pred = selection_model.predict(select_X_test)
	predictions = [round(value) for value in y_pred]
	accuracy = accuracy_score(Y_cv, predictions)
	print("Thresh=%.3f, n=%d, Accuracy: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))


Thresh=0.267, n=1, Accuracy: 85.90%
Thresh=0.134, n=2, Accuracy: 85.90%
Thresh=0.061, n=3, Accuracy: 85.94%
Thresh=0.041, n=4, Accuracy: 85.94%
Thresh=0.041, n=5, Accuracy: 85.87%
Thresh=0.040, n=6, Accuracy: 85.87%
Thresh=0.035, n=7, Accuracy: 86.67%
Thresh=0.029, n=8, Accuracy: 86.67%
Thresh=0.023, n=9, Accuracy: 86.67%
Thresh=0.019, n=10, Accuracy: 86.74%
Thresh=0.018, n=11, Accuracy: 86.74%
Thresh=0.017, n=12, Accuracy: 86.63%
Thresh=0.017, n=13, Accuracy: 86.63%
Thresh=0.014, n=14, Accuracy: 86.70%
Thresh=0.014, n=15, Accuracy: 86.70%
Thresh=0.013, n=16, Accuracy: 86.70%
Thresh=0.013, n=17, Accuracy: 86.70%
Thresh=0.011, n=18, Accuracy: 86.70%
Thresh=0.011, n=19, Accuracy: 86.63%
Thresh=0.010, n=20, Accuracy: 86.63%
Thresh=0.010, n=21, Accuracy: 86.63%
Thresh=0.010, n=22, Accuracy: 86.63%
Thresh=0.006, n=23, Accuracy: 86.63%
Thresh=0.006, n=24, Accuracy: 86.63%
Thresh=0.006, n=25, Accuracy: 86.44%
Thresh=0.005, n=26, Accuracy: 86.44%
Thresh=0.005, n=27, Accuracy: 86.67%
Thresh=0.0

# Apply Logistic Regression

In [0]:
clf = LogisticRegression(C=1) #fit Logistic regression with default parameter
clf.fit(X_train,Y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
print("Evalute based on validation set")
print("======>f1 : " + " %s" % f1_score(Y_cv, clf.predict(X_cv)))
print("======>accuracy score  :" + " %s" % accuracy_score(Y_cv, clf.predict(X_cv)))
print("======>Confusion matrix : ")
print(confusion_matrix(Y_cv, clf.predict(X_cv)))
print('======>Classification report  :')
print(classification_report(Y_cv, clf.predict(X_cv)))

Evalute based on validation set
======>f1 :  0.9210775047258978
======>accuracy score  : 0.8727619047619047
======>Confusion matrix : 
[[ 342  238]
 [  96 1949]]
======>Classification report  :
             precision    recall  f1-score   support

          0       0.78      0.59      0.67       580
          1       0.89      0.95      0.92      2045

avg / total       0.87      0.87      0.87      2625



In [0]:
# Make prediction on Test Data

In [0]:
X_test= test_df #preprocessed Testing data which we have done above
X_test

,Num_of_Product,UniqueA,UniqueB,FirstA_A00001,FirstA_A00002,FirstA_A00003,FirstA_A00004,FirstA_A00005,FirstA_A00006,FirstA_A00007,FirstA_A00008,FirstA_A00009,FirstA_A00010,FirstA_A00011,FirstB_B00001,FirstB_B00002,FirstB_B00003,FirstB_B00004,FirstB_B00005,FirstB_B00006,FirstB_B00007,FirstB_B00008,FirstB_B00009,FirstB_B00010,FirstB_B00011,FirstB_B00012,FirstB_B00013,FirstB_B00014,FirstB_B00015,FirstB_B00016,FirstB_B00017,FirstB_B00018,FirstB_B00019,FirstB_B00020,FirstB_B00021,FirstB_B00022,FirstB_B00023,FirstB_B00024,FirstB_B00025,FirstB_B00026,...,FirstB_B00059,FirstB_B00060,FirstB_B00061,FirstB_B00062,FirstB_B00063,FirstB_B00064,FirstB_B00065,FirstB_B00066,FirstB_B00067,FirstB_B00069,FirstB_B00070,FirstB_B00071,FirstB_B00072,FirstB_B00073,FirstB_B00074,FirstB_B00075,FirstB_B00076,FirstB_B00077,FirstB_B00078,FirstB_B00079,FirstB_B00080,FirstB_B00082,FirstB_B00083,FirstB_B00085,FirstB_B00088,FirstB_B00091,MostA_A00001,MostA_A00002,MostA_A00003,MostA_A00004,MostA_A00005,MostA_A00006,MostA_A00007,MostA_A00008,MostA_A00009,MostA_A00010,MostA_A00011,duration,weekday,hour_24h
10500,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,1,13
10501,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,4,13
10502,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,6,10
10503,4,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3.0,1,20
10504,1,1,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0.0,0,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,3,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1.0,0,21
14996,2,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1.0,2,7
14997,9,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,8.0,5,21
14998,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.0,2,8


In [0]:
predictions = clf.predict(X_test) # predict target feature value

In [0]:
predictions

array([1, 1, 1, ..., 1, 0, 1])

In [0]:
test.head()

,session_id,startTime,endTime,ProductList,train
0,u12112,08/12/14 13:36,08/12/14 13:36,A00002/B00003/C00006/D19956/,0
1,u19725,19/12/14 13:52,19/12/14 13:52,A00002/B00005/C00067/D02026/,0
2,u11795,01/12/14 10:44,01/12/14 10:44,A00002/B00002/C00004/D12538/,0
3,u22639,08/12/14 20:19,08/12/14 20:22,A00002/B00003/C00079/D22781/;A00002/B00003/C00...,0
4,u18034,15/12/14 19:33,15/12/14 19:33,A00002/B00001/C00010/D23419/,0


In [0]:
Session_ID= test['session_id']  # test data session id

In [0]:
submission_test=pd.DataFrame({"session id": Session_ID,"gender":predictions}) 
# dataframe with session id and gender on test.csv

In [0]:
submission_test

,session id,gender
0,u12112,1
1,u19725,1
2,u11795,1
3,u22639,1
4,u18034,1
...,...,...
4495,u23966,0
4496,u20527,1
4497,u13253,1
4498,u17094,0


In [0]:
# replacing 1 with female and 0 with male
submission_test['gender']=submission_test['gender'].map({1:'female',0:'male'})

In [0]:
submission_test # final submission dataframe

,session id,gender
0,u12112,female
1,u19725,female
2,u11795,female
3,u22639,female
4,u18034,female
...,...,...
4495,u23966,male
4496,u20527,female
4497,u13253,female
4498,u17094,male


In [0]:
#saving in csv format
submission_test.to_csv('submission_test.csv', index=False)